In [1]:
### This will look for all the autoTMA images, and check if the corresponding cores are on the server
### if not, generate and upload them
## It also generate a composite core of the H&E and the TMA core
import dagSecrets, io
import girder_client, os, random, re, tqdm
%load_ext autoreload
%autoreload 2
import tmaHelpers as tma

gc = girder_client.GirderClient(apiUrl=dagSecrets.apiUrl)
gc.authenticate(apiKey=dagSecrets.apiToken)
## Samples Images 62bb03e4e70def4ae72689f5

{'_id': '5cacfa86e62914004a60e139'}

In [2]:
part1rootFolder = '62e9287be70def4ae7275e94'

part1Items = []

for f in gc.listFolder(part1rootFolder):
    print(f['name'])
    for i in gc.listItem(f['_id']):
#         print("\t",i['name'])
        part1Items.append(i)
        
random.seed(42)
random.shuffle(part1Items)     

B1
B2
B3
B4
B5
B6
B7
B8
B9


In [3]:
part2folders =   list(gc.listFolder('639736214da1ec8c4fff4dcc'))

print(len(part1Items))
p = re.compile("(?P<slideId>\d+)\.(?P<c>[A-Za-z]+)(?P<p2>\d+).yaml")
shuffleNameMap = []

for idx,i in tqdm.tqdm(enumerate(part1Items)):
    m = p.search(i['name'])
    if m:
        newItemName = "%d.%s%d.core" % (int(m.groupdict()['slideId'])+20,m.groupdict()['c'],int(m.groupdict()['p2'])+30)
        shuffleNameMap.append("%s,--->,%s" % ( i['name'],newItemName))
        newItem = gc.post("item/%s/copy?folderId=%s&name=%s" % (i['_id'],part2folders[idx%9]['_id'],newItemName ))
        #gc.addMetadataToItem(newItem['_id'],{'origFileName': None, 'oid': i['_id']})


780


780it [00:49, 15.63it/s]


In [4]:
part2Info = []

for f in gc.listFolder('639736214da1ec8c4fff4dcc'):
    for i in gc.listItem(f['_id']):
        itemInfo = {'folderName': f['name'], 'coreName': i['name'], 'coreId': i['_id'], 
                    'folderPath' : '/RING Ki67/Part 2/%s' % (f['name'])}
        part2Info.append(itemInfo)

In [6]:
import pandas as pd

df = pd.DataFrame(part2Info)
df.to_csv("RingKI67.Study.Part2.csv")


In [ ]:
# part1rootFolder = '62e9287be70def4ae7275e94'

# for f in gc.listFolder(part1rootFolder):
#     print(f['name'])
#     for i in gc.listItem(f['_id']):
# #         print("\t",i['name'])
#         part1Items.append(i)
 

In [7]:
import pandas as pd
dfShuffle = pd.DataFrame(shuffleNameMap)
dfShuffle.to_csv("RingKi67.Part2.ShuffleMap.csv")

In [ ]:
part2folders =   list(gc.listFolder('639736214da1ec8c4fff4dcc'))

for f in part2folders:
    for i in gc.listItem(f['_id']):
        gc.delete("item/%s" % i['_id'])
#             if 'origFileName' in i['meta']:
#                 print(i)
#                 break
            
#         if '(1)' in i['name']:
#             print(i['name'])
#             gc.delete("item/%s" % i['_id'])